## `builtin`

除`zipline`标准因子、过滤器、分类器外，增加`builtin`模块，将基础数据与标准`pipeline`整合，修改或模拟`quantopian`IDE功能。主要包括以下部分：
+ 自定义因子
+ 自定义过滤器
+ 自定义分类器

In [1]:
from zipline.pipeline.fundamentals.reader import Fundamentals
from zipline.pipeline.builtin import TradingDays,QTradableStocks
from zipline.research import run_pipeline, select_output_by
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage,Returns

### 因子

#### `SuccessiveYZ`
连续一字板数量（涨停、跌停）

**注意**统计连续一字，而非期间一字总数

In [2]:
from zipline.pipeline.builtin import SuccessiveYZ


def make_pipeline():
    window_length = 100
    yzzt, yzdt = SuccessiveYZ()
    return Pipeline(columns={
        '期间涨幅': Returns(window_length=window_length),
        '涨停个数': yzzt,
        '跌停个数': yzdt
    })


result = run_pipeline(make_pipeline(), '2018-4-24', '2018-4-27')
select_output_by(result, assets=['603876', '603733'])

期间涨幅  涨停个数  跌停个数
2018-04-24 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0
2018-04-25 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0
2018-04-26 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0
2018-04-27 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0

In [3]:
result = run_pipeline(make_pipeline(), '2018-2-5', '2018-2-9')
select_output_by(result, assets=['600150', '000693', '600074'])

期间涨幅  涨停个数  跌停个数
2018-02-05 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          *ST保千(600074) -0.723773   1.0   1.0
                          *ST船舶(600150) -0.004439   1.0   1.0
2018-02-06 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          *ST保千(600074) -0.737247   1.0   1.0
                          *ST船舶(600150) -0.010826   1.0   1.0
2018-02-07 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          *ST保千(600074) -0.750722   1.0   1.0
                          *ST船舶(600150) -0.010826   1.0   1.0
2018-02-08 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          *ST保千(600074) -0.763234   1.0   1.0
                          *ST船舶(600150) -0.014776   1.0   1.0
2018-02-09 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          *ST保千(600074) -0.751684   1.0   1.0
                          *ST船舶(600150)  0.003253   1.0   1.0

#### `NDays` 上市天数

In [4]:
from zipline.pipeline.builtin import NDays

In [5]:
def make_pipeline():
    ndays = NDays()
    return Pipeline(
        columns={
            '上市天数': ndays,
        }
    )

In [6]:
result = run_pipeline(make_pipeline(), '2018-1-20', '2018-1-26')

In [7]:
select_output_by(result,assets=['600645','603103','603214','603876'])

上市天数
2018-01-22 00:00:00+00:00 中源协和(600645)  9029.0
                          横店影视(603103)   102.0
2018-01-23 00:00:00+00:00 中源协和(600645)  9030.0
                          横店影视(603103)   103.0
2018-01-24 00:00:00+00:00 中源协和(600645)  9031.0
                          横店影视(603103)   104.0
2018-01-25 00:00:00+00:00 中源协和(600645)  9032.0
                          横店影视(603103)   105.0
2018-01-26 00:00:00+00:00 中源协和(600645)  9033.0
                          横店影视(603103)   106.0

In [8]:
from zipline.pipeline.builtin import TradingDays

#### `TradingDays` 期间交易天数
+ 当天成交量大于0,有效交易
+ 需要指定`window_length`

In [9]:
def make_pipeline():
    t20 = TradingDays(window_length=20)
    t200 = TradingDays(window_length=200)
    return Pipeline(
        columns={
            '20天内有效交易天数': t20,
            '200天内有效交易天数': t200,           
        }, 
    )

In [10]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [11]:
select_output_by(result,'2018-04-23','2018-04-24',assets=['000001','600645','600076'])

20天内有效交易天数  200天内有效交易天数
2018-04-23 00:00:00+00:00 平安银行(000001)        20.0        200.0
                          康欣新材(600076)         0.0        150.0
                          中源协和(600645)        20.0        127.0
2018-04-24 00:00:00+00:00 平安银行(000001)        20.0        200.0
                          康欣新材(600076)         0.0        149.0
                          中源协和(600645)        20.0        127.0

+ 600076期间停牌三个月，20日内无交易，但200天内存在交易
+ 600645在200天内有停牌，有效成交率不足90%
+ 000001每天正常交易

#### `SuccessiveSuspensionDays`
连续停牌天数

In [12]:
from zipline.pipeline.builtin import SuccessiveSuspensionDays

def make_pipeline():
    days_90 = SuccessiveSuspensionDays(window_length=90, include=True)
    return Pipeline(
        columns={
            '90天内停牌天数': days_90,
            '成交量': USEquityPricing.volume.latest,           
        }, 
    )

result = run_pipeline(make_pipeline(), '2018-1-1', '2018-4-27')
select_output_by(result,'2018-04-23','2018-04-24',assets=['000001','600645','600076'])

90天内停牌天数          成交量
2018-04-23 00:00:00+00:00 平安银行(000001)       0.0   95860000.0
                          康欣新材(600076)      50.0          0.0
                          中源协和(600645)      31.0    6070000.0
2018-04-24 00:00:00+00:00 平安银行(000001)       0.0  107020000.0
                          康欣新材(600076)      51.0          0.0
                          中源协和(600645)      30.0    4820000.0

### 过滤器

#### `IsST`
+ 当前是否为ST状态

In [13]:
from zipline.pipeline.builtin import IsST

In [14]:
def make_pipeline():
    is_st = IsST()
    # 用于mask参数，非st计算结果为NaN
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], mask=is_st, window_length=200)
    return Pipeline(columns={
        '平均收盘': ma20,
    })

In [15]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [16]:
select_output_by(
    result, '2018-04-23', assets=['600408', '600645', '600076'])

平均收盘
2018-04-23 00:00:00+00:00 康欣新材(600076)        NaN
                          *ST安泰(600408)       NaN
                          中源协和(600645)   25.04455
2018-04-24 00:00:00+00:00 康欣新材(600076)        NaN
                          *ST安泰(600408)       NaN
                          中源协和(600645)   25.04640
2018-04-25 00:00:00+00:00 康欣新材(600076)        NaN
                          *ST安泰(600408)       NaN
                          中源协和(600645)   25.04905
2018-04-26 00:00:00+00:00 康欣新材(600076)        NaN
                          *ST安泰(600408)       NaN
                          中源协和(600645)   25.05145

In [17]:
def make_pipeline():
    is_st = IsST()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    # 用于screen参数，非st不会显示
    return Pipeline(
        columns={
            '平均收盘': ma20,
        }, 
        screen=is_st
    )

In [18]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [19]:
select_output_by(
    result, '2018-04-23', assets=['600408', '600645', '600076'])

,,平均收盘
2018-04-23 00:00:00+00:00,中源协和(600645),23.1590
2018-04-24 00:00:00+00:00,中源协和(600645),23.0225
2018-04-25 00:00:00+00:00,中源协和(600645),23.0230
2018-04-26 00:00:00+00:00,中源协和(600645),22.9925


#### `IsNewShare` 次新股
+ days：上市天数小于指定天数，判定为次新股，默认90天

In [20]:
from zipline.pipeline.builtin import IsNewShare

In [21]:
def make_pipeline():
    ndays = NDays()
    return Pipeline(
        columns={
            '上市天数': ndays,
        }, 
        screen=IsNewShare()
    )

In [22]:
run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

上市天数
2018-04-20 00:00:00+00:00 华西证券(002926)   74.0
                          泰永长征(002927)   56.0
                          华夏航空(002928)   49.0
                          润建通信(002929)   50.0
                          宏川智慧(002930)   23.0
                          锋龙股份(002931)   17.0
                          天邑股份(300504)   21.0
                          彩讯股份(300634)   28.0
                          南京聚隆(300644)   73.0
                          科顺股份(300737)   85.0
                          明阳电路(300739)   78.0
                          御家汇(300740)    71.0
                          华宝股份(300741)   50.0
                          江苏租赁(600901)   50.0
                          湖南盐业(600929)   25.0
                          成都银行(601838)   79.0
                          倍加洁(603059)    49.0
                          养元饮品(603156)   67.0
                          爱婴室(603214)    21.0
                          振德医疗(603301)    8.0
                          华菱精工(603356)   86.0
                          南都物业(603506)   78.0
                          淳中科技(603516)   77.0
                          今创集团(603680)   52.0
                          中源家居(603709)   71.0
                          XD七一二(603712)  53.0
                          沃格光电(603773)    3.0
                          嘉友国际(603871)   73.0
                          鼎胜新材(603876)    2.0
                          天永智能(603895)   88.0
...                                       ...
2018-04-25 00:00:00+00:00 长城科技(603897)   15.0
2018-04-26 00:00:00+00:00 华西证券(002926)   80.0
                          泰永长征(002927)   62.0
                          华夏航空(002928)   55.0
                          润建通信(002929)   56.0
                          宏川智慧(002930)   29.0
                          锋龙股份(002931)   23.0
                          天邑股份(300504)   27.0
                          彩讯股份(300634)   34.0
                          南京聚隆(300644)   79.0
                          明阳电路(300739)   84.0
                          御家汇(300740)    77.0
                          华宝股份(300741)   56.0
                          江苏租赁(600901)   56.0
                          湖南盐业(600929)   31.0
                          成都银行(601838)   85.0
                          倍加洁(603059)    55.0
                          养元饮品(603156)   73.0
                          爱婴室(603214)    27.0
                          振德医疗(603301)   14.0
                          南都物业(603506)   84.0
                          淳中科技(603516)   83.0
                          今创集团(603680)   58.0
                          中源家居(603709)   77.0
                          XD七一二(603712)  59.0
                          仙鹤股份(603733)    6.0
                          沃格光电(603773)    9.0
                          嘉友国际(603871)   79.0
                          鼎胜新材(603876)    8.0
                          长城科技(603897)   16.0

[152 rows x 1 columns]

#### `QTradableStocks` 量化可交易股票

In [23]:
from zipline.pipeline.builtin import QTradableStocks

In [24]:
def make_pipeline():
    stocks = QTradableStocks()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            'ma20': ma20,
        }, 
        screen=stocks
    )

In [25]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [26]:
ds = result.index.get_level_values(0).unique()

In [27]:
for d in ds:
    print('在日期为{}时，有{}只股票符合'.format(d.date(), result.loc[d].shape[0]))

在日期为2018-04-20时，有2623只股票符合
在日期为2018-04-23时，有2622只股票符合
在日期为2018-04-24时，有2626只股票符合
在日期为2018-04-25时，有2630只股票符合
在日期为2018-04-26时，有2631只股票符合


#### `TopAverageAmount` & `TAA` 平均成交额前N位
+ 平均成交额排名前N位的股票。默认前500位

In [28]:
from zipline.pipeline.builtin import TAA

def make_pipeline():
    stocks = QTradableStocks()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            'ma20': ma20,
        }, 
        screen=TAA()
    )

result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')
result.loc['2018-04-20'].shape == (500,1)

True

#### `IsYZZT` & `IsYZDT`
+ 默认包含ST一字板
+ 如排除，设定`include_st=False`

In [29]:
from zipline.pipeline.builtin import IsYZZT
from zipline.pipeline.factors import DailyReturns
def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅':dr,
            'ma20': ma20,
        }, 
        screen=IsYZZT()
    )
# 只输出一字涨停部分
result = run_pipeline(make_pipeline(), '2017-9-20', '2017-9-24')
# 注意其中包含600228、000403等ST股票
select_output_by(result, assets=['600228', '000403', '002893','300699'])

涨幅       ma20
2017-09-20 00:00:00+00:00 华通热力(002893)  0.100295  13.603333
                          光威复材(300699)  0.099935  30.569231
2017-09-21 00:00:00+00:00 华通热力(002893)  0.099866  14.305000
                          光威复材(300699)  0.100098  32.381429
2017-09-22 00:00:00+00:00 ST生化(000403)  0.050081  31.007450
                          华通热力(002893)  0.099939  15.054000
                          光威复材(300699)  0.099928  34.324667
                          ST昌九(600228)  0.049951  10.235500

In [30]:
def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅':dr,
            'ma20': ma20,
        }, 
        # 不包含ST
        screen=IsYZZT(include_st=False)
    )
# 只输出一字涨停部分
result = run_pipeline(make_pipeline(), '2017-9-20', '2017-9-24')
# 注意，此时没有包含600228、000403等ST股票
select_output_by(result, assets=['600228', '000403', '002893','300699'])

涨幅       ma20
2017-09-20 00:00:00+00:00 华通热力(002893)  0.100295  13.603333
                          光威复材(300699)  0.099935  30.569231
2017-09-21 00:00:00+00:00 华通热力(002893)  0.099866  14.305000
                          光威复材(300699)  0.100098  32.381429
2017-09-22 00:00:00+00:00 华通热力(002893)  0.099939  15.054000
                          光威复材(300699)  0.099928  34.324667

In [31]:
from zipline.pipeline.builtin import IsYZDT


def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅': dr,
            'ma20': ma20,
        },
        screen=IsYZDT())


# 只输出一字跌停部分
run_pipeline(make_pipeline(), '2018-4-24', '2018-4-26')

涨幅     ma20
2018-04-24 00:00:00+00:00 *ST华泽(000693) -0.049296   6.8845
                          *ST三维(000755) -0.049149   6.1065
                          万丰奥威(002085)  -0.099919  12.3480
                          *ST尤夫(002427) -0.050237  17.0545
                          *ST龙力(002604) -0.050000   7.8790
2018-04-25 00:00:00+00:00 *ST华泽(000693) -0.049383   6.5405
                          万丰奥威(002085)  -0.100269  12.2300
                          *ST尤夫(002427) -0.049900  16.2010
                          *ST龙力(002604) -0.050817   7.7035
                          *ST船舶(600150) -0.049708  18.3700
                          *ST罗顿(600209) -0.049757   9.5270
2018-04-26 00:00:00+00:00 *ST华泽(000693) -0.049351   6.2140
                          *ST华信(002018) -0.100186   5.3630
                          *ST尤夫(002427) -0.050420  15.3900
                          圣阳股份(002580)  -0.100358   8.3280
                          *ST龙力(002604) -0.049713   7.5150
                          *ST船舶(600150) -0.049846  18.2860
                          *ST哈空(600202) -0.049342   7.6855
                          *ST罗顿(600209) -0.049808   9.4405

In [32]:
def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅': dr,
            'ma20': ma20,
        },
        # 排除st
        screen=IsYZDT(include_st=False))


# 输出不包含St一字跌停
run_pipeline(make_pipeline(), '2018-4-24', '2018-4-26')

涨幅    ma20
2018-04-24 00:00:00+00:00 万丰奥威(002085)  -0.099919  12.348
2018-04-25 00:00:00+00:00 万丰奥威(002085)  -0.100269  12.230
2018-04-26 00:00:00+00:00 *ST华信(002018) -0.100186   5.363
                          圣阳股份(002580)  -0.100358   8.328

#### 有关停复牌
当日复牌

![停牌表](./images/20180426_list.png)

In [33]:
# 可用于限定范围
from zipline.pipeline.filters import StaticSids

In [34]:
from zipline.pipeline.builtin import IsResumed, SuccessiveSuspensionDays

def make_pipeline():
    #target = StaticSids([2163,600051])
    dr = DailyReturns()
    return Pipeline(
        columns={
            '涨幅': dr,
            '停牌天数':SuccessiveSuspensionDays(include=True)
        },
        screen=IsResumed())


# 只输出连续停牌后当日复牌的股票涨跌幅
run_pipeline(make_pipeline(), '2018-4-26', '2018-4-27')

涨幅  停牌天数
2018-04-26 00:00:00+00:00 珠海中富(000659)   0.005348   1.0
                          *ST华信(002018) -0.100186  24.0
                          圣阳股份(002580)  -0.100358  89.0
                          *ST哈空(600202) -0.049342   1.0
                          *ST狮头(600539) -0.037037  63.0
                          苏美达(600710)    0.100885   6.0
                          宁波中百(600857)   0.100284   2.0
                          星湖科技(600866)   0.101149  60.0
                          渤海汽车(600960)   0.002714  89.0
                          *ST蓝科(601798) -0.050157   1.0
2018-04-27 00:00:00+00:00 中航三鑫(002163)  -0.099476  89.0
                          艾格拉斯(002619)  -0.070000  51.0
                          宁波联合(600051)  -0.099487  70.0
                          *ST椰岛(600238) -0.050078   1.0
                          *ST信通(600289)  0.003021  12.0
                          鹏起科技(600614)  -0.100394  79.0
                          *ST友好(600778) -0.045652   1.0

### 分类器

股票分类涉及到地区、行业、概念，为简化处理，不再动态跟踪数据，而是使用静态方式简化。每次回测时，都使用最新的分类数据。如股票在2018-2-1，所处地区为上海，后由于变更注册地，2018-4-1所处地区更改为北京。在2018-2-1回测时，该股票使用的分类数据是最新数据，即上海；而2018-4-1回测时，使用的分类数据为北京。但这会造成一个问题，即在不同的时间，以同样策略回测同一期间的数据时，结果会不一致。请注意此类差异。

分类器直接使用`Fundamentals`容器类

In [35]:
def make_pipeline():
    dqfl = Fundamentals.info.region.latest
    return Pipeline(
        columns={
            'dqfl': dqfl,
        }, 
        screen=dqfl.element_of([0,1,22])
    )

In [36]:
run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

dqfl
2018-04-20 00:00:00+00:00 *ST宜化(000422)    22
                          鄂武商Ａ(000501)     22
                          长航凤凰(000520)     22
                          云南白药(000538)      1
                          沙隆达Ａ(000553)     22
                          我爱我家(000560)      1
                          天茂集团(000627)     22
                          湖北广电(000665)     22
                          盈方微(000670)      22
                          襄阳轴承(000678)     22
                          *ST双环(000707)    22
                          大冶特钢(000708)     22
                          中百集团(000759)     22
                          斯太尔(000760)      22
                          长江证券(000783)     22
                          武汉中商(000785)     22
                          云铝股份(000807)      1
                          京山轻机(000821)     22
                          启迪桑德(000826)     22
                          石化机械(000852)     22
                          三湘印象(000863)      0
                          新洋丰(000902)      22
                          福星股份(000926)     22
                          南天信息(000948)      1
                          广济药业(000952)     22
                          锡业股份(000960)      1
                          长源电力(000966)     22
                          高升控股(000971)     22
                          华工科技(000988)     22
                          中航机电(002013)     22
...                                       ...
2018-04-26 00:00:00+00:00 XD日播时(603196)     0
                          保隆科技(603197)      0
                          上海洗霸(603200)      0
                          菲林格尔(603226)      0
                          DR格尔软(603232)     0
                          上海天洋(603330)      0
                          水星家纺(603365)      0
                          亚士创能(603378)      0
                          风语筑(603466)       0
                          恒为科技(603496)      0
                          荣泰健康(603579)      0
                          艾艾精工(603580)      0
                          徕木股份(603633)      0
                          晶华新材(603683)      0
                          塞力斯(603716)      22
                          海利生物(603718)      0
                          鸣志电器(603728)      0
                          龙韵股份(603729)      0
                          泰晶科技(603738)     22
                          来伊份(603777)       0
                          华荣股份(603855)      0
                          飞科电器(603868)      0
                          数据港(603881)       0
                          XD元祖股(603886)     0
                          城地股份(603887)      0
                          天永智能(603895)      0
                          晨光文具(603899)      0
                          大理药业(603963)      1
                          康德莱(603987)       0
                          至正股份(603991)      0

[1295 rows x 1 columns]

In [37]:
# 通过查询获取分类编码的含义
Fundamentals.region_cname(0)

'上海其它'

In [38]:
Fundamentals.region_cname(22)

'湖北'